In [ ]:
"""
X2CT Data Preprocessing Module
CT와 X-ray 데이터를 전처리하고 HDF5 형식으로 저장
"""

import os
import numpy as np
import h5py
import pydicom
import SimpleITK as sitk
from scipy.ndimage import zoom
import torch
from torchvision import transforms


class CTPreprocessor:
    """CT DICOM 파일을 로드하고 전처리"""

    def __init__(self, target_spacing=[1, 1, 1], target_size=256):
        self.target_spacing = target_spacing
        self.target_size = target_size

    def load_ct_slices(self, ct_folder_path):
        """DICOM 슬라이스들을 로드"""
        slices = []
        slice_thickness = None
        pixel_spacing = None

        for filename in sorted(os.listdir(ct_folder_path)):
            if filename.endswith('.dcm'):
                filepath = os.path.join(ct_folder_path, filename)
                dicom_data = pydicom.dcmread(filepath)

                if slice_thickness is None:
                    slice_thickness = float(dicom_data.SliceThickness)
                if pixel_spacing is None:
                    pixel_spacing = [float(i) for i in dicom_data.PixelSpacing]

                slices.append(dicom_data.pixel_array)

        ct_volume = np.stack(slices, axis=0)
        original_spacing = [slice_thickness] + pixel_spacing
        return ct_volume, original_spacing

    def resample(self, image, spacing):
        """이미지를 타겟 spacing으로 리샘플링"""
        spacing = np.array(spacing)
        new_spacing = np.array(self.target_spacing)

        resize_factor = spacing / new_spacing
        new_shape = np.round(image.shape * resize_factor)
        real_resize_factor = new_shape / image.shape
        new_spacing = spacing / real_resize_factor

        image = zoom(image, real_resize_factor, mode='nearest', order=1)
        return image, new_spacing

    def crop_or_pad(self, volume, target_size):
        """볼륨을 target_size로 크롭 또는 패딩"""
        d, h, w = volume.shape

        # Depth
        if d >= target_size:
            start_d = (d - target_size) // 2
            volume = volume[start_d:start_d + target_size, :, :]
        else:
            pad_d = (target_size - d) // 2
            pad_d_end = target_size - d - pad_d
            volume = np.pad(volume, ((pad_d, pad_d_end), (0, 0), (0, 0)), mode='constant')

        # Height
        d, h, w = volume.shape
        if h >= target_size:
            start_h = (h - target_size) // 2
            volume = volume[:, start_h:start_h + target_size, :]
        else:
            pad_h = (target_size - h) // 2
            pad_h_end = target_size - h - pad_h
            volume = np.pad(volume, ((0, 0), (pad_h, pad_h_end), (0, 0)), mode='constant')

        # Width
        d, h, w = volume.shape
        if w >= target_size:
            start_w = (w - target_size) // 2
            volume = volume[:, :, start_w:start_w + target_size]
        else:
            pad_w = (target_size - w) // 2
            pad_w_end = target_size - w - pad_w
            volume = np.pad(volume, ((0, 0), (0, 0), (pad_w, pad_w_end)), mode='constant')

        return volume

    def normalize_ct(self, ct_volume, window_level=40, window_width=400):
        """CT를 HU windowing 후 [-1, 1]로 정규화"""
        # HU windowing
        min_hu = window_level - window_width // 2
        max_hu = window_level + window_width // 2

        ct_volume = np.clip(ct_volume, min_hu, max_hu)
        ct_volume = (ct_volume - min_hu) / (max_hu - min_hu)  # [0, 1]
        ct_volume = ct_volume * 2 - 1  # [-1, 1]

        return ct_volume.astype(np.float32)

    def preprocess(self, ct_folder_path):
        """전체 전처리 파이프라인"""
        # 로드
        ct_volume, original_spacing = self.load_ct_slices(ct_folder_path)
        print(f"Original shape: {ct_volume.shape}, spacing: {original_spacing}")

        # 리샘플링
        ct_volume, new_spacing = self.resample(ct_volume, original_spacing)
        print(f"After resample: {ct_volume.shape}, spacing: {new_spacing}")

        # 크롭/패딩
        ct_volume = self.crop_or_pad(ct_volume, self.target_size)
        print(f"After crop/pad: {ct_volume.shape}")

        # 정규화
        ct_volume = self.normalize_ct(ct_volume)
        print(f"After normalize: min={ct_volume.min():.2f}, max={ct_volume.max():.2f}")

        return ct_volume


class XRayPreprocessor:
    """X-ray DICOM 파일을 로드하고 전처리"""

    def __init__(self, target_size=256):
        self.target_size = target_size

    def load_xray(self, xray_path):
        """단일 X-ray DICOM 파일 로드"""
        dicom_data = pydicom.dcmread(xray_path)
        xray_image = dicom_data.pixel_array.astype(np.float32)
        return xray_image

    def resize_xray(self, xray_image):
        """X-ray를 target_size로 리사이즈"""
        from scipy.ndimage import zoom

        h, w = xray_image.shape
        zoom_factors = (self.target_size / h, self.target_size / w)
        xray_image = zoom(xray_image, zoom_factors, order=1)

        return xray_image

    def normalize_xray(self, xray_image):
        """X-ray를 [-1, 1]로 정규화"""
        xray_image = xray_image - xray_image.min()
        xray_image = xray_image / (xray_image.max() + 1e-8)  # [0, 1]
        xray_image = xray_image * 2 - 1  # [-1, 1]

        return xray_image.astype(np.float32)

    def preprocess(self, xray_path):
        """전체 전처리 파이프라인"""
        xray_image = self.load_xray(xray_path)
        print(f"Original X-ray shape: {xray_image.shape}")

        xray_image = self.resize_xray(xray_image)
        print(f"After resize: {xray_image.shape}")

        xray_image = self.normalize_xray(xray_image)
        print(f"After normalize: min={xray_image.min():.2f}, max={xray_image.max():.2f}")

        return xray_image


class HDF5DataCreator:
    """전처리된 데이터를 HDF5 형식으로 저장"""

    def __init__(self, output_path):
        self.output_path = output_path

    def create_hdf5(self, ct_volume, xray_front, xray_side, patient_id):
        """HDF5 파일 생성"""
        with h5py.File(self.output_path, 'w') as f:
            f.create_dataset('ct', data=ct_volume, compression='gzip')
            f.create_dataset('xray_front', data=xray_front, compression='gzip')
            f.create_dataset('xray_side', data=xray_side, compression='gzip')
            f.attrs['patient_id'] = patient_id
            f.attrs['ct_shape'] = ct_volume.shape
            f.attrs['xray_shape'] = xray_front.shape

        print(f"HDF5 file created: {self.output_path}")


# 사용 예시
if __name__ == "__main__":
    # CT 전처리
    ct_preprocessor = CTPreprocessor(target_size=256)
    ct_volume = ct_preprocessor.preprocess(
        '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192'
    )

    # X-ray 전처리
    xray_preprocessor = XRayPreprocessor(target_size=256)
    xray_front = xray_preprocessor.preprocess(
        '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-35511/3000923.000000-NA-62357/1-1.dcm'
    )
    xray_side = xray_preprocessor.preprocess(
        '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-35511/3000923.000000-NA-62357/1-2.dcm'
    )

    # HDF5 저장
    hdf5_creator = HDF5DataCreator('/content/drive/MyDrive/processed_data/patient_0001.h5')
    hdf5_creator.create_hdf5(ct_volume, xray_front, xray_side, 'LIDC-IDRI-0001')

    print("\n✅ 데이터 전처리 완료!")

In [ ]:
"""
X2CT Network Architectures
Generator와 Discriminator 정의 (X2CT 논문 기반)
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import functools


# ============================================================================
# Basic Building Blocks
# ============================================================================

class ResnetBlock2D(nn.Module):
    """2D Resnet Block"""

    def __init__(self, dim, norm_layer=nn.BatchNorm2d, use_dropout=False):
        super(ResnetBlock2D, self).__init__()

        layers = [
            nn.Conv2d(dim, dim, kernel_size=3, padding=1, bias=False),
            norm_layer(dim),
            nn.ReLU(True),
        ]

        if use_dropout:
            layers.append(nn.Dropout(0.5))

        layers += [
            nn.Conv2d(dim, dim, kernel_size=3, padding=1, bias=False),
            norm_layer(dim),
        ]

        self.conv_block = nn.Sequential(*layers)

    def forward(self, x):
        return x + self.conv_block(x)


class ResnetBlock3D(nn.Module):
    """3D Resnet Block"""

    def __init__(self, dim, norm_layer=nn.BatchNorm3d, use_dropout=False):
        super(ResnetBlock3D, self).__init__()

        layers = [
            nn.Conv3d(dim, dim, kernel_size=3, padding=1, bias=False),
            norm_layer(dim),
            nn.ReLU(True),
        ]

        if use_dropout:
            layers.append(nn.Dropout(0.5))

        layers += [
            nn.Conv3d(dim, dim, kernel_size=3, padding=1, bias=False),
            norm_layer(dim),
        ]

        self.conv_block = nn.Sequential(*layers)

    def forward(self, x):
        return x + self.conv_block(x)


# ============================================================================
# 2D Encoder (X-ray → 2D Features)
# ============================================================================

class Encoder2D(nn.Module):
    """
    2D Encoder for X-ray images
    입력: [B, 1, 256, 256]
    출력: [B, 512, 16, 16]
    """

    def __init__(self, input_nc=1, ngf=64, n_downsampling=4, norm_layer=nn.BatchNorm2d):
        super(Encoder2D, self).__init__()

        # Initial convolution
        model = [
            nn.Conv2d(input_nc, ngf, kernel_size=7, padding=3, bias=False),
            norm_layer(ngf),
            nn.ReLU(True)
        ]

        # Downsampling
        for i in range(n_downsampling):
            mult = 2 ** i
            model += [
                nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1, bias=False),
                norm_layer(ngf * mult * 2),
                nn.ReLU(True)
            ]

        # Resnet blocks
        mult = 2 ** n_downsampling
        for i in range(2):
            model += [ResnetBlock2D(ngf * mult, norm_layer=norm_layer)]

        self.model = nn.Sequential(*model)
        self.output_nc = ngf * mult

    def forward(self, x):
        return self.model(x)


# ============================================================================
# Connection Modules (2D → 3D dimension lifting)
# ============================================================================

class ConnectionB(nn.Module):
    """
    Connection-B: 2D feature를 3D로 확장
    [B, C, H, W] → [B, C, D, H, W]
    """

    def __init__(self, in_channels, depth=256):
        super(ConnectionB, self).__init__()
        self.depth = depth

        # 3D convolution for dimension expansion
        self.expand_conv = nn.Sequential(
            nn.Conv3d(in_channels, in_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(in_channels),
            nn.ReLU(True)
        )

    def forward(self, x):
        """
        x: [B, C, H, W]
        return: [B, C, D, H, W]
        """
        B, C, H, W = x.shape

        # Expand along depth dimension
        x = x.unsqueeze(2)  # [B, C, 1, H, W]
        x = x.repeat(1, 1, self.depth, 1, 1)  # [B, C, D, H, W]

        # Refine with 3D conv
        x = self.expand_conv(x)

        return x


class ConnectionC(nn.Module):
    """
    Connection-C: Multi-view fusion
    두 개의 직교 뷰를 3D 공간에서 융합
    """

    def __init__(self, channels):
        super(ConnectionC, self).__init__()

        self.fusion_conv = nn.Sequential(
            nn.Conv3d(channels * 2, channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(channels),
            nn.ReLU(True),
            nn.Conv3d(channels, channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(channels),
            nn.ReLU(True)
        )

    def forward(self, view1, view2):
        """
        view1: [B, C, D, H, W] - PA view
        view2: [B, C, D, H, W] - Lateral view (depth와 height가 교환됨)
        """
        # view2를 회전: depth ↔ height
        view2_rotated = view2.permute(0, 1, 3, 2, 4)  # [B, C, H, D, W]

        # 크기 맞추기
        if view1.shape != view2_rotated.shape:
            view2_rotated = F.interpolate(
                view2_rotated,
                size=view1.shape[2:],
                mode='trilinear',
                align_corners=False
            )

        # Concatenate and fuse
        fused = torch.cat([view1, view2_rotated], dim=1)
        output = self.fusion_conv(fused)

        return output


# ============================================================================
# 3D Decoder (3D Features → CT Volume)
# ============================================================================

class Decoder3D(nn.Module):
    """
    3D Decoder for CT volume generation
    입력: [B, 512, 16, 16, 16]
    출력: [B, 1, 256, 256, 256]
    """

    def __init__(self, input_nc=512, output_nc=1, ngf=64, n_upsampling=4, norm_layer=nn.BatchNorm3d):
        super(Decoder3D, self).__init__()

        model = []

        # Resnet blocks
        for i in range(2):
            model += [ResnetBlock3D(input_nc, norm_layer=norm_layer)]

        # Upsampling
        for i in range(n_upsampling):
            mult = 2 ** (n_upsampling - i)
            model += [
                nn.ConvTranspose3d(ngf * mult, int(ngf * mult / 2),
                                   kernel_size=4, stride=2, padding=1, bias=False),
                norm_layer(int(ngf * mult / 2)),
                nn.ReLU(True)
            ]

        # Output layer
        model += [
            nn.Conv3d(ngf, output_nc, kernel_size=7, padding=3),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


# ============================================================================
# Generator (Complete X2CT-GAN Generator)
# ============================================================================

class X2CTGenerator(nn.Module):
    """
    X2CT-GAN Generator
    두 개의 직교 X-ray 이미지로부터 3D CT 볼륨 생성
    """

    def __init__(self, input_nc=1, output_nc=1, ngf=64, depth=256):
        super(X2CTGenerator, self).__init__()

        # 각 뷰를 위한 2D 인코더
        self.encoder_view1 = Encoder2D(input_nc, ngf, n_downsampling=4)
        self.encoder_view2 = Encoder2D(input_nc, ngf, n_downsampling=4)

        # 2D → 3D 확장 (Connection-B)
        encoder_output_nc = ngf * 16  # 2^4 = 16
        self.connection_b_view1 = ConnectionB(encoder_output_nc, depth=depth//16)
        self.connection_b_view2 = ConnectionB(encoder_output_nc, depth=depth//16)

        # Multi-view fusion (Connection-C)
        self.connection_c = ConnectionC(encoder_output_nc)

        # 3D 디코더
        self.decoder = Decoder3D(encoder_output_nc, output_nc, ngf, n_upsampling=4)

    def forward(self, x):
        """
        x: list of [view1, view2]
        view1: [B, 1, 256, 256] - PA X-ray
        view2: [B, 1, 256, 256] - Lateral X-ray
        return: [B, 1, 256, 256, 256] - CT volume
        """
        if not isinstance(x, list) or len(x) != 2:
            raise ValueError("Input must be a list of [view1, view2]")

        view1, view2 = x

        # 2D encoding
        feat1 = self.encoder_view1(view1)  # [B, 512, 16, 16]
        feat2 = self.encoder_view2(view2)  # [B, 512, 16, 16]

        # 2D → 3D expansion
        feat1_3d = self.connection_b_view1(feat1)  # [B, 512, 16, 16, 16]
        feat2_3d = self.connection_b_view2(feat2)  # [B, 512, 16, 16, 16]

        # Multi-view fusion
        fused = self.connection_c(feat1_3d, feat2_3d)  # [B, 512, 16, 16, 16]

        # 3D decoding
        output = self.decoder(fused)  # [B, 1, 256, 256, 256]

        return output


# ============================================================================
# Discriminator (3D PatchGAN)
# ============================================================================

class X2CTDiscriminator(nn.Module):
    """
    3D PatchGAN Discriminator
    입력: [B, 1, 256, 256, 256] (CT volume only)
    """

    def __init__(self, input_nc=1, ndf=64, n_layers=3, norm_layer=nn.BatchNorm3d):
        super(X2CTDiscriminator, self).__init__()

        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm3d
        else:
            use_bias = norm_layer == nn.InstanceNorm3d

        kw = 4
        padw = 1

        sequence = [
            nn.Conv3d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw),
            nn.LeakyReLU(0.2, True)
        ]

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv3d(ndf * nf_mult_prev, ndf * nf_mult,
                          kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv3d(ndf * nf_mult_prev, ndf * nf_mult,
                      kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [
            nn.Conv3d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)
        ]

        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """
        input: [B, 1, D, H, W]
        return: [B, 1, ...]
        """
        return self.model(input)


# ============================================================================
# 테스트
# ============================================================================

if __name__ == "__main__":
    print("Testing X2CT Networks...")

    # Generator test
    generator = X2CTGenerator(input_nc=1, output_nc=1, ngf=64, depth=256)
    view1 = torch.randn(1, 1, 256, 256)
    view2 = torch.randn(1, 1, 256, 256)

    print(f"\nGenerator Input:")
    print(f"  View1: {view1.shape}")
    print(f"  View2: {view2.shape}")

    fake_ct = generator([view1, view2])
    print(f"\nGenerator Output:")
    print(f"  Fake CT: {fake_ct.shape}")

    # Discriminator test
    discriminator = X2CTDiscriminator(input_nc=1, ndf=64, n_layers=3)
    pred = discriminator(fake_ct)
    print(f"\nDiscriminator Output:")
    print(f"  Prediction: {pred.shape}")

    # Parameter count
    gen_params = sum(p.numel() for p in generator.parameters())
    disc_params = sum(p.numel() for p in discriminator.parameters())
    print(f"\n✅ Networks initialized successfully!")
    print(f"Generator parameters: {gen_params:,}")
    print(f"Discriminator parameters: {disc_params:,}")

In [ ]:
"""
X2CT Loss Functions and Dataset
GAN Loss, Reconstruction Loss, Dataset 정의
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import h5py
import numpy as np
import random


# ============================================================================
# Loss Functions
# ============================================================================

class GANLoss(nn.Module):
    """
    GAN Loss (LSGAN)
    """

    def __init__(self, use_lsgan=True, target_real_label=1.0, target_fake_label=0.0):
        super(GANLoss, self).__init__()
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))

        if use_lsgan:
            self.loss = nn.MSELoss()
        else:
            self.loss = nn.BCEWithLogitsLoss()

    def get_target_tensor(self, input, target_is_real):
        if target_is_real:
            target_tensor = self.real_label
        else:
            target_tensor = self.fake_label

        return target_tensor.expand_as(input)

    def __call__(self, input, target_is_real):
        target_tensor = self.get_target_tensor(input, target_is_real)
        loss = self.loss(input, target_tensor)
        return loss


class ReconstructionLoss(nn.Module):
    """
    Reconstruction Loss (L1 or L2)
    """

    def __init__(self, loss_type='l1'):
        super(ReconstructionLoss, self).__init__()
        if loss_type == 'l1':
            self.loss = nn.L1Loss()
        elif loss_type == 'l2':
            self.loss = nn.MSELoss()
        else:
            raise ValueError(f"Unsupported loss type: {loss_type}")

    def __call__(self, pred, target):
        return self.loss(pred, target)


class ProjectionLoss(nn.Module):
    """
    Projection Loss (optional)
    생성된 3D CT를 2D로 투영하여 입력 X-ray와 비교
    """

    def __init__(self):
        super(ProjectionLoss, self).__init__()
        self.loss = nn.L1Loss()

    def forward(self, ct_volume, xray_front, xray_side):
        """
        ct_volume: [B, 1, D, H, W]
        xray_front: [B, 1, H, W]
        xray_side: [B, 1, H, W]
        """
        # DRR simulation (simple average projection)
        # Front view (average over width dimension)
        drr_front = torch.mean(ct_volume, dim=4)  # [B, 1, D, H]
        drr_front = torch.mean(drr_front, dim=2)  # [B, 1, H]
        drr_front = drr_front.unsqueeze(2)  # [B, 1, 1, H]
        drr_front = drr_front.repeat(1, 1, xray_front.shape[2], 1)  # [B, 1, H, H]

        # Side view (average over depth dimension)
        drr_side = torch.mean(ct_volume, dim=2)  # [B, 1, H, W]

        # Resize to match X-ray size
        if drr_front.shape != xray_front.shape:
            drr_front = F.interpolate(drr_front, size=xray_front.shape[2:],
                                      mode='bilinear', align_corners=False)
        if drr_side.shape != xray_side.shape:
            drr_side = F.interpolate(drr_side, size=xray_side.shape[2:],
                                     mode='bilinear', align_corners=False)

        loss_front = self.loss(drr_front, xray_front)
        loss_side = self.loss(drr_side, xray_side)

        return loss_front + loss_side


# ============================================================================
# Image Pool (for stable training)
# ============================================================================

class ImagePool:
    """
    이미지 풀: 과거 생성된 이미지를 저장하고 랜덤으로 반환
    Discriminator 학습 안정화
    """

    def __init__(self, pool_size):
        self.pool_size = pool_size
        if self.pool_size > 0:
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        if self.pool_size == 0:
            return images

        return_images = []
        for image in images:
            image = torch.unsqueeze(image.data, 0)
            if self.num_imgs < self.pool_size:
                self.num_imgs += 1
                self.images.append(image)
                return_images.append(image)
            else:
                p = random.uniform(0, 1)
                if p > 0.5:
                    random_id = random.randint(0, self.pool_size - 1)
                    tmp = self.images[random_id].clone()
                    self.images[random_id] = image
                    return_images.append(tmp)
                else:
                    return_images.append(image)

        return_images = torch.cat(return_images, 0)
        return return_images


# ============================================================================
# Dataset
# ============================================================================

class X2CTDataset(Dataset):
    """
    X2CT Dataset for HDF5 files
    각 샘플: CT volume + 2 X-ray images
    """

    def __init__(self, data_root, file_list, phase='train', target_size=256):
        """
        Args:
            data_root: HDF5 파일들이 있는 디렉토리
            file_list: 파일 리스트 (.txt)
            phase: 'train' or 'test'
            target_size: 목표 크기
        """
        self.data_root = data_root
        self.phase = phase
        self.target_size = target_size

        # Load file list
        with open(file_list, 'r') as f:
            self.file_paths = [line.strip() for line in f.readlines()]

        print(f"Loaded {len(self.file_paths)} samples for {phase}")

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]

        # Load HDF5
        with h5py.File(file_path, 'r') as f:
            ct = f['ct'][:]
            xray_front = f['xray_front'][:]
            xray_side = f['xray_side'][:]

        # Convert to torch tensors
        ct = torch.from_numpy(ct).float().unsqueeze(0)  # [1, D, H, W]
        xray_front = torch.from_numpy(xray_front).float().unsqueeze(0)  # [1, H, W]
        xray_side = torch.from_numpy(xray_side).float().unsqueeze(0)  # [1, H, W]

        # Data augmentation (only for training)
        if self.phase == 'train':
            # Random flip
            if random.random() > 0.5:
                ct = torch.flip(ct, [3])  # Flip width
                xray_front = torch.flip(xray_front, [2])

            # Random rotation (small angle)
            # (생략 - 필요시 추가)

        return {
            'ct': ct,
            'xray_front': xray_front,
            'xray_side': xray_side,
            'path': file_path
        }


class SimpleX2CTDataset(Dataset):
    """
    Simplified X2CT Dataset for single sample testing
    단일 샘플로 빠른 테스트용
    """

    def __init__(self, ct_volume, xray_front, xray_side):
        """
        Args:
            ct_volume: numpy array [D, H, W]
            xray_front: numpy array [H, W]
            xray_side: numpy array [H, W]
        """
        self.ct = torch.from_numpy(ct_volume).float().unsqueeze(0)
        self.xray_front = torch.from_numpy(xray_front).float().unsqueeze(0)
        self.xray_side = torch.from_numpy(xray_side).float().unsqueeze(0)

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return {
            'ct': self.ct,
            'xray_front': self.xray_front,
            'xray_side': self.xray_side,
            'path': 'single_sample'
        }


# ============================================================================
# Metrics
# ============================================================================

def compute_psnr(img1, img2, max_val=1.0):
    """Peak Signal-to-Noise Ratio"""
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 20 * torch.log10(max_val / torch.sqrt(mse))
    return psnr.item()


def compute_mse(img1, img2):
    """Mean Squared Error"""
    mse = torch.mean((img1 - img2) ** 2)
    return mse.item()


def compute_mae(img1, img2):
    """Mean Absolute Error"""
    mae = torch.mean(torch.abs(img1 - img2))
    return mae.item()


def compute_ssim(img1, img2, window_size=11):
    """
    Structural Similarity Index (simplified version)
    실제 사용시 pytorch-msssim 패키지 권장
    """
    # Simple implementation
    mu1 = torch.mean(img1)
    mu2 = torch.mean(img2)

    sigma1_sq = torch.var(img1)
    sigma2_sq = torch.var(img2)
    sigma12 = torch.mean((img1 - mu1) * (img2 - mu2))

    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    ssim = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / \
           ((mu1 ** 2 + mu2 ** 2 + C1) * (sigma1_sq + sigma2_sq + C2))

    return ssim.item()


# ============================================================================
# 테스트
# ============================================================================

if __name__ == "__main__":
    print("Testing Loss Functions and Dataset...")

    # Loss functions test
    gan_loss = GANLoss(use_lsgan=True)
    recon_loss = ReconstructionLoss(loss_type='l1')
    proj_loss = ProjectionLoss()

    # Dummy data
    fake_pred = torch.randn(2, 1, 8, 8, 8)
    real_pred = torch.randn(2, 1, 8, 8, 8)
    fake_ct = torch.randn(2, 1, 256, 256, 256)
    real_ct = torch.randn(2, 1, 256, 256, 256)
    xray_front = torch.randn(2, 1, 256, 256)
    xray_side = torch.randn(2, 1, 256, 256)

    # Test losses
    loss_gan_fake = gan_loss(fake_pred, False)
    loss_gan_real = gan_loss(real_pred, True)
    loss_recon = recon_loss(fake_ct, real_ct)
    loss_proj = proj_loss(fake_ct, xray_front, xray_side)

    print(f"\nLoss values:")
    print(f"  GAN loss (fake): {loss_gan_fake.item():.4f}")
    print(f"  GAN loss (real): {loss_gan_real.item():.4f}")
    print(f"  Reconstruction loss: {loss_recon.item():.4f}")
    print(f"  Projection loss: {loss_proj.item():.4f}")

    # Test metrics
    psnr = compute_psnr(fake_ct, real_ct)
    mse = compute_mse(fake_ct, real_ct)
    mae = compute_mae(fake_ct, real_ct)
    ssim = compute_ssim(fake_ct, real_ct)

    print(f"\nMetrics:")
    print(f"  PSNR: {psnr:.2f} dB")
    print(f"  MSE: {mse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  SSIM: {ssim:.4f}")

    print(f"\n✅ Loss functions and metrics working!")

In [ ]:
"""
X2CTGAN Model
Generator와 Discriminator를 통합한 전체 GAN 모델
"""

import torch
import torch.nn as nn
from torch.optim import Adam
import os


class X2CTGAN(nn.Module):
    """
    X2CT-GAN Model
    Generator와 Discriminator를 관리하고 학습/추론 수행
    """

    def __init__(self, opt, generator, discriminator):
        super(X2CTGAN, self).__init__()
        self.opt = opt
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Networks
        self.netG = generator.to(self.device)
        self.netD = discriminator.to(self.device)

        # Loss functions (import from losses_and_dataset.py)
        from losses_and_dataset import GANLoss, ReconstructionLoss, ProjectionLoss

        self.criterionGAN = GANLoss(use_lsgan=True).to(self.device)
        self.criterionL1 = ReconstructionLoss(loss_type='l1').to(self.device)
        self.criterionProj = ProjectionLoss().to(self.device) if opt.use_proj_loss else None

        # Optimizers
        self.optimizer_G = Adam(
            self.netG.parameters(),
            lr=opt.lr_G,
            betas=(opt.beta1, opt.beta2)
        )
        self.optimizer_D = Adam(
            self.netD.parameters(),
            lr=opt.lr_D,
            betas=(opt.beta1, opt.beta2)
        )

        # Image pool for discriminator
        from losses_and_dataset import ImagePool
        self.fake_pool = ImagePool(opt.pool_size)

        # For storing losses
        self.loss_D = 0
        self.loss_G_GAN = 0
        self.loss_G_L1 = 0
        self.loss_G_Proj = 0
        self.loss_G = 0

    def set_input(self, input_data):
        """입력 데이터 설정"""
        self.xray_front = input_data['xray_front'].to(self.device)
        self.xray_side = input_data['xray_side'].to(self.device)
        self.ct_real = input_data['ct'].to(self.device)

    def forward(self):
        """Generator forward pass"""
        self.ct_fake = self.netG([self.xray_front, self.xray_side])

    def backward_D(self):
        """Discriminator backward pass"""
        # Real CT
        pred_real = self.netD(self.ct_real)
        loss_D_real = self.criterionGAN(pred_real, True)

        # Fake CT
        ct_fake = self.fake_pool.query(self.ct_fake)
        pred_fake = self.netD(ct_fake.detach())
        loss_D_fake = self.criterionGAN(pred_fake, False)

        # Combined loss
        self.loss_D = (loss_D_real + loss_D_fake) * 0.5
        self.loss_D.backward()

    def backward_G(self):
        """Generator backward pass"""
        # GAN loss
        pred_fake = self.netD(self.ct_fake)
        self.loss_G_GAN = self.criterionGAN(pred_fake, True) * self.opt.lambda_GAN

        # Reconstruction loss (L1)
        self.loss_G_L1 = self.criterionL1(self.ct_fake, self.ct_real) * self.opt.lambda_L1

        # Projection loss (optional)
        if self.criterionProj is not None:
            self.loss_G_Proj = self.criterionProj(
                self.ct_fake,
                self.xray_front,
                self.xray_side
            ) * self.opt.lambda_Proj
        else:
            self.loss_G_Proj = 0

        # Total generator loss
        self.loss_G = self.loss_G_GAN + self.loss_G_L1 + self.loss_G_Proj
        self.loss_G.backward()

    def optimize_parameters(self):
        """한 스텝 최적화"""
        # Forward
        self.forward()

        # Update G
        self.set_requires_grad(self.netD, False)
        self.optimizer_G.zero_grad()
        self.backward_G()
        self.optimizer_G.step()

        # Update D
        self.set_requires_grad(self.netD, True)
        self.optimizer_D.zero_grad()
        self.backward_D()
        self.optimizer_D.step()

    def set_requires_grad(self, nets, requires_grad=False):
        """네트워크의 gradient 계산 on/off"""
        if not isinstance(nets, list):
            nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

    def get_current_losses(self):
        """현재 loss 값들 반환"""
        return {
            'D': self.loss_D.item() if isinstance(self.loss_D, torch.Tensor) else self.loss_D,
            'G_GAN': self.loss_G_GAN.item(),
            'G_L1': self.loss_G_L1.item(),
            'G_Proj': self.loss_G_Proj.item() if isinstance(self.loss_G_Proj, torch.Tensor) else 0,
            'G': self.loss_G.item()
        }

    def get_current_visuals(self):
        """시각화를 위한 이미지들 반환"""
        return {
            'xray_front': self.xray_front,
            'xray_side': self.xray_side,
            'ct_real': self.ct_real,
            'ct_fake': self.ct_fake
        }

    def save_networks(self, epoch, save_dir):
        """네트워크 저장"""
        os.makedirs(save_dir, exist_ok=True)

        save_path_G = os.path.join(save_dir, f'netG_epoch_{epoch}.pth')
        save_path_D = os.path.join(save_dir, f'netD_epoch_{epoch}.pth')

        torch.save({
            'epoch': epoch,
            'state_dict': self.netG.state_dict(),
            'optimizer': self.optimizer_G.state_dict(),
        }, save_path_G)

        torch.save({
            'epoch': epoch,
            'state_dict': self.netD.state_dict(),
            'optimizer': self.optimizer_D.state_dict(),
        }, save_path_D)

        print(f"✅ Networks saved at epoch {epoch}")

    def load_networks(self, epoch, save_dir):
        """네트워크 로드"""
        load_path_G = os.path.join(save_dir, f'netG_epoch_{epoch}.pth')
        load_path_D = os.path.join(save_dir, f'netD_epoch_{epoch}.pth')

        if os.path.exists(load_path_G):
            checkpoint_G = torch.load(load_path_G, map_location=self.device)
            self.netG.load_state_dict(checkpoint_G['state_dict'])
            self.optimizer_G.load_state_dict(checkpoint_G['optimizer'])
            print(f"✅ Generator loaded from epoch {epoch}")

        if os.path.exists(load_path_D):
            checkpoint_D = torch.load(load_path_D, map_location=self.device)
            self.netD.load_state_dict(checkpoint_D['state_dict'])
            self.optimizer_D.load_state_dict(checkpoint_D['optimizer'])
            print(f"✅ Discriminator loaded from epoch {epoch}")

    def eval(self):
        """평가 모드"""
        self.netG.eval()
        self.netD.eval()

    def train(self):
        """학습 모드"""
        self.netG.train()
        self.netD.train()

    def inference(self, xray_front, xray_side):
        """
        추론 (학습 없이 CT 생성)
        Args:
            xray_front: [B, 1, H, W]
            xray_side: [B, 1, H, W]
        Returns:
            ct_fake: [B, 1, D, H, W]
        """
        self.eval()
        with torch.no_grad():
            xray_front = xray_front.to(self.device)
            xray_side = xray_side.to(self.device)
            ct_fake = self.netG([xray_front, xray_side])
        return ct_fake


# ============================================================================
# Configuration
# ============================================================================

class Options:
    """학습 옵션"""

    def __init__(self):
        # Network
        self.input_nc = 1
        self.output_nc = 1
        self.ngf = 64
        self.ndf = 64
        self.n_layers_D = 3
        self.depth = 256

        # Loss weights
        self.lambda_GAN = 1.0
        self.lambda_L1 = 100.0
        self.lambda_Proj = 10.0
        self.use_proj_loss = False  # DRR 없이는 False

        # Optimizer
        self.lr_G = 0.0002
        self.lr_D = 0.0002
        self.beta1 = 0.5
        self.beta2 = 0.999

        # Training
        self.pool_size = 50
        self.batch_size = 1
        self.n_epochs = 100
        self.n_epochs_decay = 100

        # Directories
        self.checkpoints_dir = '/content/drive/MyDrive/x2ct_checkpoints'
        self.save_epoch_freq = 5

        # Display
        self.print_freq = 10


# ============================================================================
# 테스트
# ============================================================================

if __name__ == "__main__":
    print("Testing X2CTGAN Model...")

    # Import networks
    from networks import X2CTGenerator, X2CTDiscriminator

    # Options
    opt = Options()

    # Initialize networks
    generator = X2CTGenerator(
        input_nc=opt.input_nc,
        output_nc=opt.output_nc,
        ngf=opt.ngf,
        depth=opt.depth
    )

    discriminator = X2CTDiscriminator(
        input_nc=opt.output_nc,
        ndf=opt.ndf,
        n_layers=opt.n_layers_D
    )

    # Initialize model
    model = X2CTGAN(opt, generator, discriminator)

    # Test forward pass
    dummy_data = {
        'xray_front': torch.randn(1, 1, 256, 256),
        'xray_side': torch.randn(1, 1, 256, 256),
        'ct': torch.randn(1, 1, 256, 256, 256)
    }

    model.set_input(dummy_data)
    model.optimize_parameters()

    # Get losses
    losses = model.get_current_losses()
    print(f"\nLosses after one optimization step:")
    for name, value in losses.items():
        print(f"  {name}: {value:.4f}")

    # Test inference
    xray_front = torch.randn(1, 1, 256, 256)
    xray_side = torch.randn(1, 1, 256, 256)
    ct_fake = model.inference(xray_front, xray_side)
    print(f"\nInference output shape: {ct_fake.shape}")

    print(f"\n✅ X2CTGAN model working perfectly!")

In [ ]:
"""
X2CT Training Script
완전한 학습 루프 구현
"""

import torch
from torch.utils.data import DataLoader
import time
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np


class Trainer:
    """X2CTGAN Trainer"""

    def __init__(self, model, train_dataset, val_dataset, opt):
        self.model = model
        self.opt = opt

        # Data loaders
        self.train_loader = DataLoader(
            train_dataset,
            batch_size=opt.batch_size,
            shuffle=True,
            num_workers=0,  # Colab에서는 0으로 설정
            pin_memory=True if torch.cuda.is_available() else False
        )

        self.val_loader = DataLoader(
            val_dataset,
            batch_size=1,
            shuffle=False,
            num_workers=0
        ) if val_dataset is not None else None

        # For tracking
        self.total_steps = 0
        self.epoch_start_time = time.time()

        # Loss history
        self.loss_history = {
            'D': [],
            'G_GAN': [],
            'G_L1': [],
            'G': []
        }

        # Create checkpoint directory
        os.makedirs(opt.checkpoints_dir, exist_ok=True)

    def train_epoch(self, epoch):
        """한 에폭 학습"""
        self.model.train()
        epoch_losses = {'D': 0, 'G_GAN': 0, 'G_L1': 0, 'G': 0}

        pbar = tqdm(self.train_loader, desc=f'Epoch {epoch}')
        for i, data in enumerate(pbar):
            self.total_steps += 1

            # Set input and optimize
            self.model.set_input(data)
            self.model.optimize_parameters()

            # Get losses
            losses = self.model.get_current_losses()

            # Accumulate
            for key in epoch_losses:
                if key in losses:
                    epoch_losses[key] += losses[key]

            # Update progress bar
            pbar.set_postfix({
                'D': f"{losses['D']:.4f}",
                'G': f"{losses['G']:.4f}",
                'G_L1': f"{losses['G_L1']:.4f}"
            })

            # Print detailed losses
            if self.total_steps % self.opt.print_freq == 0:
                self.print_current_losses(epoch, i, losses)

        # Average losses
        n_batches = len(self.train_loader)
        for key in epoch_losses:
            epoch_losses[key] /= n_batches
            self.loss_history[key].append(epoch_losses[key])

        return epoch_losses

    def validate(self, epoch):
        """검증"""
        if self.val_loader is None:
            return None

        self.model.eval()
        val_losses = {'D': 0, 'G_GAN': 0, 'G_L1': 0, 'G': 0}

        with torch.no_grad():
            for i, data in enumerate(tqdm(self.val_loader, desc='Validation')):
                self.model.set_input(data)
                self.model.forward()

                # Compute losses without backward
                pred_fake = self.model.netD(self.model.ct_fake)
                pred_real = self.model.netD(self.model.ct_real)

                loss_G_GAN = self.model.criterionGAN(pred_fake, True)
                loss_G_L1 = self.model.criterionL1(self.model.ct_fake, self.model.ct_real)
                loss_D_fake = self.model.criterionGAN(pred_fake, False)
                loss_D_real = self.model.criterionGAN(pred_real, True)

                val_losses['G_GAN'] += loss_G_GAN.item()
                val_losses['G_L1'] += loss_G_L1.item()
                val_losses['D'] += ((loss_D_fake + loss_D_real) * 0.5).item()
                val_losses['G'] += (loss_G_GAN + loss_G_L1 * self.opt.lambda_L1).item()

        # Average
        n_batches = len(self.val_loader)
        for key in val_losses:
            val_losses[key] /= n_batches

        print(f"\n{'='*60}")
        print(f"Validation Results at Epoch {epoch}:")
        for key, value in val_losses.items():
            print(f"  Val_{key}: {value:.4f}")
        print(f"{'='*60}\n")

        return val_losses

    def print_current_losses(self, epoch, iters, losses):
        """현재 loss 출력"""
        message = f'Epoch: {epoch}, Iters: {iters}, '
        for k, v in losses.items():
            message += f'{k}: {v:.4f} '
        print(message)

    def save_checkpoint(self, epoch):
        """체크포인트 저장"""
        self.model.save_networks(epoch, self.opt.checkpoints_dir)

    def plot_losses(self, save_path=None):
        """Loss 그래프 저장"""
        plt.figure(figsize=(12, 8))

        plt.subplot(2, 2, 1)
        plt.plot(self.loss_history['D'])
        plt.title('Discriminator Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)

        plt.subplot(2, 2, 2)
        plt.plot(self.loss_history['G'])
        plt.title('Generator Total Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)

        plt.subplot(2, 2, 3)
        plt.plot(self.loss_history['G_GAN'], label='GAN Loss')
        plt.plot(self.loss_history['G_L1'], label='L1 Loss')
        plt.title('Generator Loss Components')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.subplot(2, 2, 4)
        plt.plot(self.loss_history['G_L1'])
        plt.title('Reconstruction Loss (L1)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"Loss plot saved to {save_path}")

        plt.show()

    def visualize_results(self, epoch, num_samples=1):
        """결과 시각화"""
        self.model.eval()

        with torch.no_grad():
            for i, data in enumerate(self.val_loader if self.val_loader else self.train_loader):
                if i >= num_samples:
                    break

                self.model.set_input(data)
                self.model.forward()

                visuals = self.model.get_current_visuals()

                # Convert to numpy
                xray_front = visuals['xray_front'][0, 0].cpu().numpy()
                xray_side = visuals['xray_side'][0, 0].cpu().numpy()
                ct_real = visuals['ct_real'][0, 0].cpu().numpy()
                ct_fake = visuals['ct_fake'][0, 0].cpu().numpy()

                # Plot
                fig, axes = plt.subplots(2, 4, figsize=(16, 8))

                # X-rays
                axes[0, 0].imshow(xray_front, cmap='gray')
                axes[0, 0].set_title('Input X-ray (Front)')
                axes[0, 0].axis('off')

                axes[0, 1].imshow(xray_side, cmap='gray')
                axes[0, 1].set_title('Input X-ray (Side)')
                axes[0, 1].axis('off')

                # Real CT slices
                mid_slice = ct_real.shape[0] // 2
                axes[0, 2].imshow(ct_real[mid_slice], cmap='gray')
                axes[0, 2].set_title('Real CT (Mid Slice)')
                axes[0, 2].axis('off')

                axes[0, 3].imshow(ct_real[:, mid_slice, :], cmap='gray')
                axes[0, 3].set_title('Real CT (Coronal)')
                axes[0, 3].axis('off')

                # Fake CT slices
                axes[1, 0].text(0.5, 0.5, f'Epoch {epoch}',
                               ha='center', va='center', fontsize=16)
                axes[1, 0].axis('off')

                axes[1, 1].axis('off')

                axes[1, 2].imshow(ct_fake[mid_slice], cmap='gray')
                axes[1, 2].set_title('Generated CT (Mid Slice)')
                axes[1, 2].axis('off')

                axes[1, 3].imshow(ct_fake[:, mid_slice, :], cmap='gray')
                axes[1, 3].set_title('Generated CT (Coronal)')
                axes[1, 3].axis('off')

                plt.tight_layout()

                save_path = os.path.join(
                    self.opt.checkpoints_dir,
                    f'results_epoch_{epoch}_sample_{i}.png'
                )
                plt.savefig(save_path, dpi=150, bbox_inches='tight')
                print(f"Results saved to {save_path}")

                plt.show()

    def train(self, start_epoch=0):
        """전체 학습 루프"""
        print(f"\n{'='*60}")
        print(f"Starting Training")
        print(f"Total Epochs: {self.opt.n_epochs}")
        print(f"Batch Size: {self.opt.batch_size}")
        print(f"Device: {self.model.device}")
        print(f"{'='*60}\n")

        for epoch in range(start_epoch, self.opt.n_epochs):
            epoch_start_time = time.time()

            # Train
            train_losses = self.train_epoch(epoch)

            # Validate
            if self.val_loader is not None and epoch % 5 == 0:
                val_losses = self.validate(epoch)

            # Print epoch summary
            epoch_time = time.time() - epoch_start_time
            print(f"\nEpoch {epoch} completed in {epoch_time:.2f}s")
            print(f"Train Losses: D={train_losses['D']:.4f}, "
                  f"G={train_losses['G']:.4f}, G_L1={train_losses['G_L1']:.4f}")

            # Save checkpoint
            if (epoch + 1) % self.opt.save_epoch_freq == 0:
                self.save_checkpoint(epoch)
                self.plot_losses(
                    os.path.join(self.opt.checkpoints_dir, f'loss_plot_epoch_{epoch}.png')
                )

            # Visualize results
            if (epoch + 1) % self.opt.save_epoch_freq == 0:
                self.visualize_results(epoch, num_samples=2)

        print(f"\n{'='*60}")
        print(f"Training Completed!")
        print(f"{'='*60}\n")

        # Final save
        self.save_checkpoint(self.opt.n_epochs - 1)
        self.plot_losses(
            os.path.join(self.opt.checkpoints_dir, 'final_loss_plot.png')
        )


# ============================================================================
# Main Training Function
# ============================================================================

def main():
    """메인 학습 함수"""

    # Import필요한 모듈들
    from networks import X2CTGenerator, X2CTDiscriminator
    from model import X2CTGAN, Options
    from losses_and_dataset import SimpleX2CTDataset
    import numpy as np

    print("="*60)
    print("X2CT-GAN Training")
    print("="*60)

    # Options
    opt = Options()
    opt.n_epochs = 50  # 테스트용으로 50 에폭
    opt.save_epoch_freq = 5
    opt.batch_size = 1
    opt.use_proj_loss = False  # DRR 없으면 False

    # Create networks
    print("\n1. Initializing networks...")
    generator = X2CTGenerator(
        input_nc=opt.input_nc,
        output_nc=opt.output_nc,
        ngf=opt.ngf,
        depth=opt.depth
    )

    discriminator = X2CTDiscriminator(
        input_nc=opt.output_nc,
        ndf=opt.ndf,
        n_layers=opt.n_layers_D
    )

    # Create model
    print("2. Creating X2CTGAN model...")
    model = X2CTGAN(opt, generator, discriminator)

    # Create dummy dataset (실제 사용시 HDF5 데이터셋으로 교체)
    print("3. Loading dataset...")
    # 여기서는 테스트용 더미 데이터 사용
    dummy_ct = np.random.randn(256, 256, 256).astype(np.float32)
    dummy_xray_front = np.random.randn(256, 256).astype(np.float32)
    dummy_xray_side = np.random.randn(256, 256).astype(np.float32)

    train_dataset = SimpleX2CTDataset(dummy_ct, dummy_xray_front, dummy_xray_side)
    val_dataset = SimpleX2CTDataset(dummy_ct, dummy_xray_front, dummy_xray_side)

    # Create trainer
    print("4. Creating trainer...")
    trainer = Trainer(model, train_dataset, val_dataset, opt)

    # Start training
    print("\n5. Starting training...")
    trainer.train(start_epoch=0)

    print("\n✅ Training completed successfully!")


if __name__ == "__main__":
    main()

In [ ]:
"""
X2CT Testing and Inference Script
학습된 모델로 추론 및 평가
"""

import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os


class Tester:
    """X2CTGAN Tester"""

    def __init__(self, model, test_dataset, opt):
        self.model = model
        self.opt = opt
        self.test_dataset = test_dataset

        from torch.utils.data import DataLoader
        self.test_loader = DataLoader(
            test_dataset,
            batch_size=1,
            shuffle=False,
            num_workers=0
        )

        # Metrics
        self.metrics = {
            'PSNR': [],
            'MSE': [],
            'MAE': [],
            'SSIM': []
        }

    def compute_metrics(self, pred, target):
        """메트릭 계산"""
        from losses_and_dataset import compute_psnr, compute_mse, compute_mae, compute_ssim

        psnr = compute_psnr(pred, target)
        mse = compute_mse(pred, target)
        mae = compute_mae(pred, target)
        ssim = compute_ssim(pred, target)

        return {
            'PSNR': psnr,
            'MSE': mse,
            'MAE': mae,
            'SSIM': ssim
        }

    def test(self, visualize=True):
        """테스트 실행"""
        print(f"\n{'='*60}")
        print(f"Starting Testing")
        print(f"{'='*60}\n")

        self.model.eval()

        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                print(f"\nProcessing sample {i+1}/{len(self.test_loader)}...")

                # Set input and forward
                self.model.set_input(data)
                self.model.forward()

                # Get results
                visuals = self.model.get_current_visuals()
                ct_real = visuals['ct_real']
                ct_fake = visuals['ct_fake']

                # Compute metrics
                metrics = self.compute_metrics(ct_fake, ct_real)

                # Store metrics
                for key, value in metrics.items():
                    self.metrics[key].append(value)

                # Print metrics
                print(f"Metrics for sample {i+1}:")
                for key, value in metrics.items():
                    print(f"  {key}: {value:.4f}")

                # Visualize
                if visualize:
                    self.visualize_sample(
                        visuals,
                        metrics,
                        sample_idx=i,
                        save_path=os.path.join(
                            self.opt.checkpoints_dir,
                            f'test_result_sample_{i}.png'
                        )
                    )

        # Print average metrics
        print(f"\n{'='*60}")
        print(f"Average Metrics:")
        for key in self.metrics:
            avg = np.mean(self.metrics[key])
            std = np.std(self.metrics[key])
            print(f"  {key}: {avg:.4f} ± {std:.4f}")
        print(f"{'='*60}\n")

        return self.metrics

    def visualize_sample(self, visuals, metrics, sample_idx=0, save_path=None):
        """샘플 시각화 (다양한 뷰)"""
        # Convert to numpy
        xray_front = visuals['xray_front'][0, 0].cpu().numpy()
        xray_side = visuals['xray_side'][0, 0].cpu().numpy()
        ct_real = visuals['ct_real'][0, 0].cpu().numpy()
        ct_fake = visuals['ct_fake'][0, 0].cpu().numpy()

        # Create figure
        fig = plt.figure(figsize=(20, 12))

        # 1. Input X-rays
        ax1 = plt.subplot(3, 5, 1)
        ax1.imshow(xray_front, cmap='gray')
        ax1.set_title('Input X-ray (Front View)', fontsize=12)
        ax1.axis('off')

        ax2 = plt.subplot(3, 5, 2)
        ax2.imshow(xray_side, cmap='gray')
        ax2.set_title('Input X-ray (Side View)', fontsize=12)
        ax2.axis('off')

        # 2. Real CT - Different slices
        mid_d = ct_real.shape[0] // 2
        mid_h = ct_real.shape[1] // 2
        mid_w = ct_real.shape[2] // 2

        ax3 = plt.subplot(3, 5, 3)
        ax3.imshow(ct_real[mid_d, :, :], cmap='gray')
        ax3.set_title('Real CT (Axial)', fontsize=12)
        ax3.axis('off')

        ax4 = plt.subplot(3, 5, 4)
        ax4.imshow(ct_real[:, mid_h, :], cmap='gray')
        ax4.set_title('Real CT (Coronal)', fontsize=12)
        ax4.axis('off')

        ax5 = plt.subplot(3, 5, 5)
        ax5.imshow(ct_real[:, :, mid_w], cmap='gray')
        ax5.set_title('Real CT (Sagittal)', fontsize=12)
        ax5.axis('off')

        # 3. Fake CT - Different slices
        ax6 = plt.subplot(3, 5, 8)
        ax6.imshow(ct_fake[mid_d, :, :], cmap='gray')
        ax6.set_title('Generated CT (Axial)', fontsize=12)
        ax6.axis('off')

        ax7 = plt.subplot(3, 5, 9)
        ax7.imshow(ct_fake[:, mid_h, :], cmap='gray')
        ax7.set_title('Generated CT (Coronal)', fontsize=12)
        ax7.axis('off')

        ax8 = plt.subplot(3, 5, 10)
        ax8.imshow(ct_fake[:, :, mid_w], cmap='gray')
        ax8.set_title('Generated CT (Sagittal)', fontsize=12)
        ax8.axis('off')

        # 4. Difference maps
        diff = np.abs(ct_real - ct_fake)

        ax9 = plt.subplot(3, 5, 13)
        im = ax9.imshow(diff[mid_d, :, :], cmap='hot')
        ax9.set_title('Difference (Axial)', fontsize=12)
        ax9.axis('off')
        plt.colorbar(im, ax=ax9, fraction=0.046)

        ax10 = plt.subplot(3, 5, 14)
        im = ax10.imshow(diff[:, mid_h, :], cmap='hot')
        ax10.set_title('Difference (Coronal)', fontsize=12)
        ax10.axis('off')
        plt.colorbar(im, ax=ax10, fraction=0.046)

        ax11 = plt.subplot(3, 5, 15)
        im = ax11.imshow(diff[:, :, mid_w], cmap='hot')
        ax11.set_title('Difference (Sagittal)', fontsize=12)
        ax11.axis('off')
        plt.colorbar(im, ax=ax11, fraction=0.046)

        # 5. Metrics text
        ax_text = plt.subplot(3, 5, 6)
        ax_text.axis('off')
        metrics_text = f"Sample {sample_idx+1} Metrics:\n\n"
        for key, value in metrics.items():
            metrics_text += f"{key}: {value:.4f}\n"
        ax_text.text(0.1, 0.5, metrics_text, fontsize=14,
                    verticalalignment='center', family='monospace')

        plt.suptitle(f'X2CT-GAN Test Results - Sample {sample_idx+1}',
                     fontsize=16, fontweight='bold')
        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"  Visualization saved to {save_path}")

        plt.show()

    def save_3d_volume(self, ct_volume, save_path):
        """3D 볼륨을 .npy 파일로 저장"""
        np.save(save_path, ct_volume)
        print(f"3D volume saved to {save_path}")


def inference_single_sample(model, xray_front, xray_side, visualize=True):
    """
    단일 샘플 추론

    Args:
        model: 학습된 X2CTGAN 모델
        xray_front: [1, 256, 256] numpy array
        xray_side: [1, 256, 256] numpy array
        visualize: 시각화 여부

    Returns:
        ct_fake: [1, 256, 256, 256] numpy array
    """
    model.eval()

    # Convert to torch tensors
    xray_front_tensor = torch.from_numpy(xray_front).float().unsqueeze(0).unsqueeze(0)
    xray_side_tensor = torch.from_numpy(xray_side).float().unsqueeze(0).unsqueeze(0)

    # Inference
    with torch.no_grad():
        ct_fake = model.inference(xray_front_tensor, xray_side_tensor)

    # Convert to numpy
    ct_fake_np = ct_fake[0, 0].cpu().numpy()

    if visualize:
        # Quick visualization
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))

        # Inputs
        axes[0, 0].imshow(xray_front, cmap='gray')
        axes[0, 0].set_title('Input: Front X-ray')
        axes[0, 0].axis('off')

        axes[0, 1].imshow(xray_side, cmap='gray')
        axes[0, 1].set_title('Input: Side X-ray')
        axes[0, 1].axis('off')

        axes[0, 2].axis('off')

        # Output CT slices
        mid = ct_fake_np.shape[0] // 2

        axes[1, 0].imshow(ct_fake_np[mid, :, :], cmap='gray')
        axes[1, 0].set_title('Output: CT (Axial)')
        axes[1, 0].axis('off')

        axes[1, 1].imshow(ct_fake_np[:, mid, :], cmap='gray')
        axes[1, 1].set_title('Output: CT (Coronal)')
        axes[1, 1].axis('off')

        axes[1, 2].imshow(ct_fake_np[:, :, mid], cmap='gray')
        axes[1, 2].set_title('Output: CT (Sagittal)')
        axes[1, 2].axis('off')

        plt.suptitle('X2CT-GAN Inference Result', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

    return ct_fake_np


# ============================================================================
# Main Test Function
# ============================================================================

def main():
    """메인 테스트 함수"""

    # Import 필요한 모듈들
    from networks import X2CTGenerator, X2CTDiscriminator
    from model import X2CTGAN, Options
    from losses_and_dataset import SimpleX2CTDataset
    import numpy as np

    print("="*60)
    print("X2CT-GAN Testing")
    print("="*60)

    # Options
    opt = Options()

    # Create networks
    print("\n1. Initializing networks...")
    generator = X2CTGenerator(
        input_nc=opt.input_nc,
        output_nc=opt.output_nc,
        ngf=opt.ngf,
        depth=opt.depth
    )

    discriminator = X2CTDiscriminator(
        input_nc=opt.output_nc,
        ndf=opt.ndf,
        n_layers=opt.n_layers_D
    )

    # Create model
    print("2. Creating X2CTGAN model...")
    model = X2CTGAN(opt, generator, discriminator)

    # Load trained model (if available)
    # model.load_networks(epoch=50, save_dir=opt.checkpoints_dir)

    # Create test dataset (실제 사용시 HDF5 데이터셋으로 교체)
    print("3. Loading test dataset...")
    dummy_ct = np.random.randn(256, 256, 256).astype(np.float32)
    dummy_xray_front = np.random.randn(256, 256).astype(np.float32)
    dummy_xray_side = np.random.randn(256, 256).astype(np.float32)

    test_dataset = SimpleX2CTDataset(dummy_ct, dummy_xray_front, dummy_xray_side)

    # Create tester
    print("4. Creating tester...")
    tester = Tester(model, test_dataset, opt)

    # Run test
    print("\n5. Running test...")
    metrics = tester.test(visualize=True)

    print("\n✅ Testing completed successfully!")


if __name__ == "__main__":
    main()

In [ ]:
"""
X2CT-GAN Complete Pipeline
사용자 데이터로 전체 파이프라인 실행

사용법:
1. Google Colab에서 실행
2. Google Drive 마운트
3. 데이터 경로 설정
4. 전체 파이프라인 실행
"""

import sys
import os

# Add current directory to path
sys.path.append('/home/claude/x2ct_complete')

import torch
import numpy as np


# ============================================================================
# STEP 1: 데이터 전처리
# ============================================================================

def step1_preprocess_data():
    """데이터 전처리 및 HDF5 저장"""
    print("\n" + "="*60)
    print("STEP 1: Data Preprocessing")
    print("="*60)

    from data_preprocessing import CTPreprocessor, XRayPreprocessor, HDF5DataCreator

    # 경로 설정 (사용자의 실제 경로로 변경)
    ct_folder = '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192'
    xray_front_path = '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-35511/3000923.000000-NA-62357/1-1.dcm'
    xray_side_path = '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-35511/3000923.000000-NA-62357/1-2.dcm'
    output_dir = '/content/drive/MyDrive/x2ct_processed_data'

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # CT 전처리
    print("\n1.1 Processing CT volume...")
    ct_preprocessor = CTPreprocessor(target_size=256)
    ct_volume = ct_preprocessor.preprocess(ct_folder)

    # X-ray 전처리
    print("\n1.2 Processing X-ray images...")
    xray_preprocessor = XRayPreprocessor(target_size=256)
    xray_front = xray_preprocessor.preprocess(xray_front_path)
    xray_side = xray_preprocessor.preprocess(xray_side_path)

    # HDF5 저장
    print("\n1.3 Saving to HDF5...")
    hdf5_path = os.path.join(output_dir, 'patient_0001.h5')
    hdf5_creator = HDF5DataCreator(hdf5_path)
    hdf5_creator.create_hdf5(ct_volume, xray_front, xray_side, 'LIDC-IDRI-0001')

    print(f"\n✅ Step 1 완료! 데이터 저장 위치: {hdf5_path}")

    return ct_volume, xray_front, xray_side, hdf5_path


# ============================================================================
# STEP 2: 네트워크 초기화
# ============================================================================

def step2_initialize_networks():
    """네트워크 초기화"""
    print("\n" + "="*60)
    print("STEP 2: Network Initialization")
    print("="*60)

    from networks import X2CTGenerator, X2CTDiscriminator
    from model import X2CTGAN, Options

    # Options
    opt = Options()
    opt.n_epochs = 20  # 테스트용으로 20 에폭
    opt.save_epoch_freq = 5
    opt.batch_size = 1
    opt.use_proj_loss = False
    opt.checkpoints_dir = '/content/drive/MyDrive/x2ct_checkpoints'

    # 디렉토리 생성
    os.makedirs(opt.checkpoints_dir, exist_ok=True)

    # Networks
    print("\n2.1 Creating Generator...")
    generator = X2CTGenerator(
        input_nc=opt.input_nc,
        output_nc=opt.output_nc,
        ngf=opt.ngf,
        depth=opt.depth
    )
    print(f"   Generator parameters: {sum(p.numel() for p in generator.parameters()):,}")

    print("\n2.2 Creating Discriminator...")
    discriminator = X2CTDiscriminator(
        input_nc=opt.output_nc,
        ndf=opt.ndf,
        n_layers=opt.n_layers_D
    )
    print(f"   Discriminator parameters: {sum(p.numel() for p in discriminator.parameters()):,}")

    # Model
    print("\n2.3 Creating X2CTGAN model...")
    model = X2CTGAN(opt, generator, discriminator)

    print(f"\n✅ Step 2 완료! Device: {model.device}")

    return model, opt


# ============================================================================
# STEP 3: 학습
# ============================================================================

def step3_train(model, opt, ct_volume, xray_front, xray_side):
    """모델 학습"""
    print("\n" + "="*60)
    print("STEP 3: Training")
    print("="*60)

    from train import Trainer
    from losses_and_dataset import SimpleX2CTDataset

    # Dataset 생성
    print("\n3.1 Creating datasets...")
    train_dataset = SimpleX2CTDataset(ct_volume, xray_front, xray_side)
    val_dataset = SimpleX2CTDataset(ct_volume, xray_front, xray_side)

    print(f"   Train samples: {len(train_dataset)}")
    print(f"   Val samples: {len(val_dataset)}")

    # Trainer 생성
    print("\n3.2 Creating trainer...")
    trainer = Trainer(model, train_dataset, val_dataset, opt)

    # 학습 시작
    print("\n3.3 Starting training...")
    trainer.train(start_epoch=0)

    print(f"\n✅ Step 3 완료! 체크포인트 저장 위치: {opt.checkpoints_dir}")

    return trainer


# ============================================================================
# STEP 4: 테스트 및 추론
# ============================================================================

def step4_test(model, opt, ct_volume, xray_front, xray_side):
    """모델 테스트 및 추론"""
    print("\n" + "="*60)
    print("STEP 4: Testing and Inference")
    print("="*60)

    from test import Tester, inference_single_sample
    from losses_and_dataset import SimpleX2CTDataset

    # Test dataset
    print("\n4.1 Creating test dataset...")
    test_dataset = SimpleX2CTDataset(ct_volume, xray_front, xray_side)

    # Tester
    print("\n4.2 Running comprehensive test...")
    tester = Tester(model, test_dataset, opt)
    metrics = tester.test(visualize=True)

    # Single inference
    print("\n4.3 Running single inference...")
    ct_generated = inference_single_sample(
        model,
        xray_front,
        xray_side,
        visualize=True
    )

    # 결과 저장
    save_path = os.path.join(opt.checkpoints_dir, 'generated_ct.npy')
    np.save(save_path, ct_generated)
    print(f"\n✅ Step 4 완료! 생성된 CT 저장 위치: {save_path}")

    return ct_generated, metrics


# ============================================================================
# STEP 5: 결과 시각화
# ============================================================================

def step5_visualize_results(ct_real, ct_generated, xray_front, xray_side):
    """최종 결과 시각화"""
    print("\n" + "="*60)
    print("STEP 5: Final Visualization")
    print("="*60)

    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # 2D 슬라이스 비교
    print("\n5.1 Creating 2D slice comparisons...")
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))

    mid_d = ct_real.shape[0] // 2
    mid_h = ct_real.shape[1] // 2
    mid_w = ct_real.shape[2] // 2

    # Inputs
    axes[0, 0].imshow(xray_front, cmap='gray')
    axes[0, 0].set_title('Input: Front X-ray', fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')

    axes[0, 1].imshow(xray_side, cmap='gray')
    axes[0, 1].set_title('Input: Side X-ray', fontsize=14, fontweight='bold')
    axes[0, 1].axis('off')

    axes[0, 2].axis('off')
    axes[0, 3].axis('off')

    # Real CT
    axes[1, 0].imshow(ct_real[mid_d, :, :], cmap='gray')
    axes[1, 0].set_title('Real CT: Axial', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(ct_real[:, mid_h, :], cmap='gray')
    axes[1, 1].set_title('Real CT: Coronal', fontsize=12)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(ct_real[:, :, mid_w], cmap='gray')
    axes[1, 2].set_title('Real CT: Sagittal', fontsize=12)
    axes[1, 2].axis('off')

    axes[1, 3].text(0.5, 0.5, 'Ground Truth', ha='center', va='center',
                   fontsize=16, fontweight='bold')
    axes[1, 3].axis('off')

    # Generated CT
    axes[2, 0].imshow(ct_generated[mid_d, :, :], cmap='gray')
    axes[2, 0].set_title('Generated CT: Axial', fontsize=12)
    axes[2, 0].axis('off')

    axes[2, 1].imshow(ct_generated[:, mid_h, :], cmap='gray')
    axes[2, 1].set_title('Generated CT: Coronal', fontsize=12)
    axes[2, 1].axis('off')

    axes[2, 2].imshow(ct_generated[:, :, mid_w], cmap='gray')
    axes[2, 2].set_title('Generated CT: Sagittal', fontsize=12)
    axes[2, 2].axis('off')

    axes[2, 3].text(0.5, 0.5, 'Generated', ha='center', va='center',
                   fontsize=16, fontweight='bold', color='blue')
    axes[2, 3].axis('off')

    plt.suptitle('X2CT-GAN: Complete 3D Reconstruction Results',
                 fontsize=18, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # 3D 볼륨 렌더링 (간단한 버전)
    print("\n5.2 Creating 3D volume rendering...")
    fig = plt.figure(figsize=(16, 6))

    # Real CT 3D
    ax1 = fig.add_subplot(121, projection='3d')

    # 샘플링 (메모리 절약)
    step = 8
    x, y, z = np.mgrid[0:ct_real.shape[0]:step,
                       0:ct_real.shape[1]:step,
                       0:ct_real.shape[2]:step]

    values = ct_real[::step, ::step, ::step]
    colors = plt.cm.gray(values / values.max())

    ax1.voxels(values > 0.5, facecolors=colors, edgecolor='none', alpha=0.5)
    ax1.set_title('Real CT (3D)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')

    # Generated CT 3D
    ax2 = fig.add_subplot(122, projection='3d')

    values_gen = ct_generated[::step, ::step, ::step]
    colors_gen = plt.cm.gray(values_gen / values_gen.max())

    ax2.voxels(values_gen > 0.5, facecolors=colors_gen, edgecolor='none', alpha=0.5)
    ax2.set_title('Generated CT (3D)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')

    plt.suptitle('3D Volume Rendering', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print(f"\n✅ Step 5 완료!")


# ============================================================================
# 전체 파이프라인 실행
# ============================================================================

def run_complete_pipeline():
    """전체 파이프라인 한번에 실행"""
    print("\n" + "="*80)
    print(" "*20 + "X2CT-GAN COMPLETE PIPELINE")
    print("="*80)
    print("\n이 스크립트는 다음을 수행합니다:")
    print("1. 데이터 전처리 (CT + X-rays)")
    print("2. 네트워크 초기화 (Generator + Discriminator)")
    print("3. GAN 학습")
    print("4. 테스트 및 추론")
    print("5. 결과 시각화")
    print("\n" + "="*80)

    try:
        # Step 1: 데이터 전처리
        ct_volume, xray_front, xray_side, hdf5_path = step1_preprocess_data()

        # Step 2: 네트워크 초기화
        model, opt = step2_initialize_networks()

        # Step 3: 학습
        trainer = step3_train(model, opt, ct_volume, xray_front, xray_side)

        # Step 4: 테스트
        ct_generated, metrics = step4_test(model, opt, ct_volume, xray_front, xray_side)

        # Step 5: 최종 시각화
        step5_visualize_results(ct_volume, ct_generated, xray_front, xray_side)

        # 최종 요약
        print("\n" + "="*80)
        print(" "*25 + "🎉 파이프라인 완료! 🎉")
        print("="*80)
        print(f"\n최종 결과:")
        print(f"  - HDF5 데이터: {hdf5_path}")
        print(f"  - 체크포인트: {opt.checkpoints_dir}")
        print(f"  - 생성된 CT shape: {ct_generated.shape}")
        print(f"\n최종 메트릭:")
        for key, values in metrics.items():
            print(f"  {key}: {np.mean(values):.4f}")
        print("\n" + "="*80)

        return model, ct_generated, metrics

    except Exception as e:
        print(f"\n❌ 에러 발생: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None


# ============================================================================
# Main
# ============================================================================

if __name__ == "__main__":
    # Google Colab 환경 확인
    try:
        import google.colab
        IN_COLAB = True
        print("✅ Google Colab 환경 감지")

        # Drive 마운트
        from google.colab import drive
        drive.mount('/content/drive')

    except:
        IN_COLAB = False
        print("⚠️  로컬 환경에서 실행 중")

    # 전체 파이프라인 실행
    model, ct_generated, metrics = run_complete_pipeline()

    if model is not None:
        print("\n✅ 모든 작업이 성공적으로 완료되었습니다!")
        print("\n다음 단계:")
        print("1. 생성된 CT 볼륨을 확인하세요")
        print("2. 체크포인트를 사용하여 새로운 X-ray로 추론하세요")
        print("3. 더 많은 데이터로 재학습하세요")
    else:
        print("\n❌ 파이프라인 실행 중 문제가 발생했습니다")
        print("   위의 에러 메시지를 확인하세요")